# Importing Libraries

In [ ]:
!pip install textblob


In [ ]:
import pandas as pd
import re
from textblob import TextBlob
import random 
from random import seed, sample, randint
from random import random
import requests
import json
import time
from datetime import datetime
import traceback
import json
from google.colab import drive

now = datetime.now()

# Accessing UnClean Data

In [ ]:
url_p1="https://raw.githubusercontent.com/junkuse/steam_data/main/games"

In [ ]:
sets=[]
for i in range(9):
  sets.append(pd.read_csv(url_p1+str(i+1)+'.csv'))


In [ ]:
final = pd.concat(sets, ignore_index=True)
final.columns

Index(['url', 'name', 'categories', 'img_url', 'user_reviews', 'all_reviews',
       'date', 'developer', 'publisher', 'price', 'requirements', 'popu_tags',
       'desc', 'full_desc', 'pegi', 'pegi_url'],
      dtype='object')

# Functions for Cleaning Data

In [ ]:
def ret_price(stringg):
  q2=""
  row=stringg
  row=re.sub(r'[^\x00-\x7f]',r'', row)
  if (row.find("$")!=-1):
      count=0
      for i in row:
          if(count==row.find("$")+1 or count==row.find("$")+2 or count==row.find("$")+3 or count==row.find("$")+4):
              if i.isdigit():
                  q2=q2+str(i)
          count=count+1
  else:
      q2="free"
  return q2

In [ ]:
def desc_getter(stringg):
  desc=""
  d1={}
  q2=""
  sort=""
  row=stringg
  row=re.sub(r'[^\x00-\x7f]',r'', row)
  if(len(row)>10):
      if(row[11]=='G'):
          sort="game"
      elif(row[11]=='C'):
          sort="content"
      elif(row[11]=='b'):
          sort="bundle"
  count=0
  for k in row:
      if(sort=="game" and count==15):
            desc=desc+" "+k
      elif(sort=="content" and count==18):
          desc=desc+" "+k
      else:
          desc=desc+k
      count=count+1
  d1["sort"]=sort
  d1["desc"]=desc
  return(d1)

In [ ]:
#sys req
rows=final[:10]['requirements']
def reqs_getter(stringg):
  row=stringg
  values=[]
  values=([m.start() for m in re.finditer('Minimum:OS:', row)])+([m.start() for m in re.finditer('Recommended:OS:', row)])
  values.sort()
  j=0
  i=0
  d2={}
  d3={}
  d4={}
  for value1 in values:
      data=""
      d1={}
      if(i!=len(values)-1):
          value2=values[i+1]
          for j in row[value1:value2]:
              data=data+j
      else:
          for j in row[value1:len(row)]:
              data=data+j
      i=i+1
      w1=data.find("Processor")
      w2=data.find("Memory")
      w3=data.find("Graphics")
      w4=data.find("Storage")
      w5=data.find("DirectX")
      w6=data.find("Minimum:OS:")
      w7=data.find("Recommended:OS:")
      count=0
      processor=""
      memory=""
      graphics=""
      minimum_os=""
      recommended_os=""
      for d in data:
          if(w2==-1):
              if(count>w1+9 and count<w1+20 and w1!=-1):
                  processor=processor+d
          else:
              if(count>w1+9 and count<w2 and w1!=-1):
                  processor=processor+d
          if(w3==-1):
              if(count>w2+6 and count<w2+10 and w2!=-1):
                  memory=memory+d
          else:
              if(count>w2+6 and count<w3 and w2!=-1):
                  memory=memory+d
          if(w5==-1):
              if(count>w3+8 and count<w4 and w3!=-1):
                  graphics=graphics+d
          else:
              if(count>w3+8 and count<w5 and w3!=-1):
                  graphics=graphics+d
          if(w6!=-1):
              if(w1!=-1):
                  if(count>w6+10 and count<w1):
                      minimum_os=minimum_os+d
              else:
                  if(count>w6+10 and count<w6+20):
                      minimum_os=minimum_os+d
          else:
              if(w1!=-1):
                  if(count>w7+15 and count<w1):
                      recommended_os=recommended_os+d
              else:
                  if(count>w7+11 and count<w7+20):
                      recommended_os=recommended_os+d       
          count=count+1
      d1["processor"]=processor
      d1["memory"]=memory
      d1["graphics"]=graphics
      if(len(minimum_os)==0):
          d1["os"]=recommended_os
          if(recommended_os.find("Windows")!=-1):
              d4["windows"]=d1
          elif(recommended_os.find("macOS")!=-1 or recommended_os.find("Mac Os")!=-1):
              d4["macOS"]=d1
          else:
              d4["linux"]=d1
      else:
          d1["os"]=minimum_os
          if(minimum_os.find("Windows")!=-1):
              d2["windows"]=d1
          elif(minimum_os.find("macOS")!=-1 or minimum_os.find("Mac Os")!=-1):
              d2["macOS"]=d1
          else:
              d2["linux"]=d1
      d3["minimum"]=d2
      d3["recommended"]=d4
  return(d3)
l1=[]
for row in rows:

    l1.append(reqs_getter(row))

# Cleaning Data and Storing in Dictionary

In [ ]:
#dictionary maker
final_dict=[]
test_data=final
pegii_list=[]
pricee_list=[]
game_id=[]
nameee=[]
for i in range(len(test_data['url'])):
  temp_dict=dict({})
  if (test_data['img_url'][i]!='-'):
    temp_dict['img_url']=test_data['img_url'][i]

  if (test_data['date'][i]!='-'):
    temp_dict['date']=test_data['date'][i]

  if (test_data['developer'][i]!='-'):
    temp_dict['developer']=test_data['developer'][i]

  if (test_data['publisher'][i]!='-'):
    temp_dict['publisher']=test_data['publisher'][i]

  if (test_data['pegi'][i]!='-'):
    try:
      asd=[re.sub(r"([A-Z][a-z]+)([A-Z][a-z]+)", r"\1 \2",i) for i in test_data['pegi'][i].split(' ')]
      asd=' '.join(asd)
      asd=asd.split(' ')
      if (test_data['pegi_url'][i]!='-'):
        temp_dict['pegi']={'pegi_url':test_data['pegi_url'][i], 'pegi_tags':asd, 'age_limit':final['pegi_url'][10].split('/')[-1].split('.')[0]}
        pegii_list.append(final['pegi_url'][10].split('/')[-1].split('.')[0])
      else:
        temp_dict['pegi']={'pegi_tags':asd, 'age_limit':final['pegi_url'][10].split('/')[-1].split('.')[0]}
        pegii_list.append(final['pegi_url'][10].split('/')[-1].split('.')[0])
    except:
      pass
    

  if (test_data['full_desc'][i]!='-'):
    temp_dict['full_desc']=desc_getter(test_data['full_desc'][i])

  if (test_data['requirements'][i]!='-'):
    temp_dict['requirements']=reqs_getter(test_data['requirements'][i])

  if (test_data['popu_tags'][i]!='-'):
    asd=re.findall('[A-Z][^A-Z]*',test_data['popu_tags'][i])
    asd=' '.join(asd)
    asd=re.sub(r'\b\w{1,3}\b', '',asd).split(' ')
    while("" in asd) : 
        asd.remove("")
    while("-" in asd) : 
        asd.remove("-")
    temp_dict['popu_tags']=asd


  if (test_data['price'][i] != '-'):
    temp_dict['price']=ret_price(test_data['price'][i])
    pricee_list.append(ret_price(test_data['price'][i]))
  temp_dict['url_info']={'url':test_data['url'][i],'id':test_data['url'][i].split('/')[4],
                        'type':test_data['url'][i].split('/')[3],
                        'url_name': ' '.join((test_data['url'][i].split('/')[5]).split('_'))}
  game_id.append(test_data['url'][i].split('/')[4])
  nameee.append(' '.join((test_data['url'][i].split('/')[5]).split('_')))
  if (test_data['name'][i]!='-'):
    temp_dict['name']=test_data['name'][i]
  if (test_data['categories'][i]!='-'):
    try:
      asd=[re.sub(r"([A-Z][a-z]+)([A-Z][a-z]+)", r"\1 \2",i) for i in test_data['categories'][i].split(' ')]
      yy=[]
      for i in range(0,30,2):
        try:
          yy.append(asd[i]+' '+asd[i+1])
        except:
            pass
      temp_dict['categories']=yy
    except:
      pass
    final_dict.append(temp_dict)


# Exporting Dictionary as JSON and storing it in Google Drive

In [ ]:
listt=final_dict
with open('game_review_data.json', 'w') as outfile:
    json.dump(listt, outfile)
drive.mount('/gdrive', force_remount=True)
with open('/gdrive/My Drive/game_review_data.json', 'w') as outfile:
  json.dump(listt, outfile)